# DTSA-5510 Unsupervised Algorithms in Machine Learning - Final Project

In this project, I'm doing an analysis of bird songs based on the [bird song data set](https://www.kaggle.com/datasets/vinayshanbhag/bird-song-data-set) available on Kaggle. This data set consists of 9110 .wav audio files of bird song recordings from 5 different bird species. I will be using spectral audio processing libraries along with dimensionality reduction and clustering algorithms to build an unsupervised model for categorizing the bird songs by species.

In [ ]:
import numpy as np
import pandas as pd

## Exploratory Data Analysis

We'll begin by importing the 